In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w4-p2/sample.csv
/kaggle/input/2021-ai-w4-p2/train.csv
/kaggle/input/2021-ai-w4-p2/test.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
train_data=pd.read_csv("/kaggle/input/2021-ai-w4-p2/train.csv")
test_data=pd.read_csv("/kaggle/input/2021-ai-w4-p2/test.csv")
sample_data=pd.read_csv("/kaggle/input/2021-ai-w4-p2/sample.csv")
print(train_data.head())
print(test_data.info())
print(sample_data.head())

    N    P    K  temperature   humidity        ph    rainfall        label
0  22   36   16    30.581395  50.771481  8.184229   64.585596    mothbeans
1  18   27   41    22.365094  92.308824  7.175344  104.821633  pomegranate
2  61   68   50    35.214628  91.497251  6.793245  243.074507       papaya
3  26   72   22    28.767949  37.577921  4.674942   91.720849   pigeonpeas
4   2  140  197    22.697801  92.822234  5.534567  105.050823        apple
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            550 non-null    int64  
 1   P            550 non-null    int64  
 2   K            550 non-null    int64  
 3   temperature  550 non-null    float64
 4   humidity     550 non-null    float64
 5   ph           550 non-null    float64
 6   rainfall     550 non-null    float64
dtypes: float64(4), int64(3)
memory usage: 30.2 KB
None
   id   labe

In [5]:
print(train_data.iloc[:,:-1].min())
print(train_data.iloc[:,:-1].max())
print(test_data.iloc[:,:].min())
print(test_data.iloc[:,:].max())

N               0.000000
P               5.000000
K               5.000000
temperature     8.825675
humidity       14.273280
ph              3.504752
rainfall       20.360011
dtype: float64
N              140.000000
P              145.000000
K              205.000000
temperature     43.675493
humidity        99.981876
ph               9.935091
rainfall       298.560117
dtype: float64
N               0.000000
P               5.000000
K               5.000000
temperature     9.467960
humidity       14.258040
ph              3.510404
rainfall       20.211267
dtype: float64
N              140.000000
P              145.000000
K              205.000000
temperature     43.360515
humidity        99.724010
ph               9.679241
rainfall       295.924880
dtype: float64


In [6]:
xtrain=train_data.iloc[: , :-1]
# xtrain=torch.FloatTensor(np.array(train_data.iloc[: , :-1]))#.cuda()
ytrain, enocde_class= pd.factorize(train_data['label'])
# ytrain=torch.LongTensor(ytrain)#.cuda()

print(xtrain[:5])
print(enocde_class)
print(ytrain[:5])

    N    P    K  temperature   humidity        ph    rainfall
0  22   36   16    30.581395  50.771481  8.184229   64.585596
1  18   27   41    22.365094  92.308824  7.175344  104.821633
2  61   68   50    35.214628  91.497251  6.793245  243.074507
3  26   72   22    28.767949  37.577921  4.674942   91.720849
4   2  140  197    22.697801  92.822234  5.534567  105.050823
Index(['mothbeans', 'pomegranate', 'papaya', 'pigeonpeas', 'apple',
       'kidneybeans', 'orange', 'rice', 'mungbean', 'watermelon', 'coconut',
       'jute', 'lentil', 'chickpea', 'cotton', 'grapes', 'mango', 'muskmelon',
       'maize', 'coffee', 'banana', 'blackgram'],
      dtype='object')
[0 1 2 3 4]


In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(xtrain)
xtrain = scaler.transform(xtrain)
print(xtrain)

[[-7.74026344e-01 -5.22604462e-01 -6.28724908e-01 ... -9.51256909e-01
   2.19982609e+00 -6.86012983e-01]
 [-8.81256962e-01 -7.93767155e-01 -1.37942559e-01 ...  9.39856699e-01
   9.02028907e-01  4.18192366e-02]
 [ 2.71472181e-01  4.41529556e-01  3.87390868e-02 ...  9.02907360e-01
   4.10508789e-01  2.54268422e+00]
 ...
 [-1.06891054e+00 -5.22604462e-01 -3.34255499e-01 ... -9.05356554e-01
   1.70580458e-03 -4.07223532e-02]
 [-1.06891054e+00  2.12876409e+00  3.06195836e+00 ...  3.82480441e-01
  -1.45336714e-01 -5.62739020e-01]
 [-5.05949799e-01  7.42821437e-01 -5.10937144e-01 ... -4.23855858e-01
   6.85524273e-01 -8.88186049e-01]]


In [8]:

xtrain=torch.FloatTensor(np.array(xtrain)).cuda()

ytrain=torch.LongTensor(ytrain).cuda()

In [9]:
import torch.nn.functional as F  # for softmax

# 모델 초기화
nb_class = 22
nb_data = len(ytrain)

W = torch.zeros((7, nb_class), requires_grad=True, device="cuda")
b = torch.zeros(1, requires_grad=True, device="cuda")

# optimizer 설정
optimizer = optim.Adam([W, b], lr=1)
nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(xtrain.matmul(W) + b, dim=1) # or .mm or @
    
    # cost 
    y_one_hot = torch.zeros(nb_data, nb_class).cuda()
    y_one_hot.scatter_(1, ytrain.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 3.091043
Epoch 1000/10000 Cost: 2.186612
Epoch 2000/10000 Cost: 2.184547
Epoch 3000/10000 Cost: 2.183713
Epoch 4000/10000 Cost: 2.183257
Epoch 5000/10000 Cost: 2.182951
Epoch 6000/10000 Cost: 2.182714
Epoch 7000/10000 Cost: 2.182523
Epoch 8000/10000 Cost: 2.182361
Epoch 9000/10000 Cost: 2.182251
Epoch 10000/10000 Cost: 2.182173


In [10]:
hypothesis = F.softmax(xtrain.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)

print(hypothesis)


# 정확도 계산 
correct_prediction = predict.float() == ytrain
print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))


tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 1.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00,  ..., 0.0000e+00, 1.4013e-45,
         0.0000e+00],
        ...,
        [6.2560e-12, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         9.3345e-24]], device='cuda:0', grad_fn=<SoftmaxBackward>)
tensor([True, True, True,  ..., True, True, True], device='cuda:0')
The model has an accuracy of 98.42% for the training set.


In [11]:
xtest = torch.FloatTensor(scaler.transform(np.array(test_data))).cuda()
hypothesis = F.softmax(xtest.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1).cuda()
ytest=sample_data['label'].values
i=0
for idx in predict:
    ytest[i]=enocde_class[idx]
    i+=1
print(ytest[:5])

['muskmelon' 'watermelon' 'papaya' 'papaya' 'apple']


In [12]:
sample_data['label']=ytest
print(sample_data.head())

   id       label
0   0   muskmelon
1   1  watermelon
2   2      papaya
3   3      papaya
4   4       apple


In [13]:
sample_data.to_csv("submission.csv",mode='w',index=False)